# CMSC 320 FINAL PROJECT: WHAT COUNTRY HAS THE GREATEST NATIONAL FOOTBALL(SOCCER) TEAM?
Owners: Nathaniel Bekele and Mikias Atnafu

## Introduction:
First and foremost, we will be establishing what we believe to be the most important boundary for this project, we will be referring to the sport exclusively as football not soccer.

The data we will be using was compiled by Mart Jürisoo and is available on Kaggle: https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017?resource=download.

## Factors to be considered
### Time period
This dataset has all the results of international football fixtures from 1872 to 2017. While it would be amazing to consider the data from 1872, we also thought of the possibility that it would be rather unfair on most countries that were colonized at the time and hence did not have a national team to compete with. Thus we will be limiting the data to 1976 to 2017 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
football_pd = pd.read_csv('results.csv')
football_pd.head()
# Create column Year using the year in the date column
football_pd['Year'] = football_pd['date'].str.extract('(\d\d\d\d)')
# Change column Year to type int
football_pd['Year'] = football_pd['Year'].astype(int)
# drop all rows with data being before 1975
football_pd = football_pd[football_pd['Year'] >= 1975]
football_pd.head()


TypeError: '>=' not supported between instances of 'str' and 'int'